In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import Libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

Reading Data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/EFQM/session 5/Air_pollution.csv', index_col=['No'])
df.head()

Check data

In [ ]:
df.info()

Fill pm2.5 Feature

In [ ]:
nan_feature = df['pm2.5'].copy().dropna()
mean = nan_feature.mean()
df['pm2.5'].fillna(mean, inplace=True)
df.info()

create dummy data for cbwd feature

In [ ]:
dummy_cwd = pd.get_dummies(df['cbwd'])
df = pd.concat([df, dummy_cwd], axis=1)
df = df.drop(columns=['cbwd'])
df.info()

Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
X = df.drop(columns=['TEMP'])
scaler = StandardScaler()
scaled_data = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)
scaled_data['TEMP'] = df['TEMP']
scaled_data.head()

Generate Sequential Data

In [ ]:
X = []
y = []

sequence_length = 4
for i in range(sequence_length, len(scaled_data)): # Windowing
    X.append(scaled_data.iloc[i-sequence_length:i].values)
    y.append(scaled_data.iloc[i]['TEMP'])

X = np.array(X)
y = np.array(y)

X, y = np.array(X), np.array(y)
X.shape, y.shape

ُSpilt dataset into train, test & Validation set

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
X_train.shape, X_val.shape, X_test.shape

Model Implementation

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(128, input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, y_train, epochs=32, batch_size=32, validation_data=(X_val, y_val),
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])

Model Training Diagram

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

Model Information

In [ ]:
model.summary()